In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import random
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from itertools import product
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
import math
import re
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import torch
from argparse import Namespace


from util.util_data import *
from util.util_dirs import *
from util.util_train import *
from ml.models import Train, VAE

from torch.utils.data import DataLoader
from torchvision import datasets

In [ ]:
inj = glob.glob('/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/injections2/fc5_coron_injections_train/*fc5.npy')
no_inj = glob.glob('/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/injections2/fc5_coron_injections_train/*[!fc5].npy')

In [ ]:
len(inj)

In [ ]:
len(no_inj)

In [ ]:

train_paths = glob.glob(os.path.join(INJECTIONS,'fc5_coron_injections_train/*.npy'))
print(len(train_paths))


syndata        = SynDatasetLabel(image_paths=train_paths)
syndata_loader = DataLoader(dataset=syndata, batch_size=32, shuffle=True)

In [ ]:
batch, labels = next(iter(syndata_loader))

In [ ]:
batch.shape

In [ ]:
labels.shape

In [ ]:
args = {
    'device':'cuda:0',
    'idx':'idx/s',
    'lr':3e-4,
    'batch_size':128,   
    'model':'vae' 
}
args = Namespace(**args)

kernels_enc = [7,5,3,5,3,5]
paddings_enc= [1,1,1,1,1,1]
strides_enc = [2,1,1,1,1,2]

maxpool = [0,0,0,0,0,0,0]

kernels_dec = [5,3,5,3,5,7]
paddings_dec= [1,1,1,1,1,1]
strides_dec = [2,1,1,1,1,2]


convdim_outputs = calculate_conv_dims(28,paddings_enc,kernels_enc,strides_enc,maxpool)

convtrans_outputs = calculate_convtrans_dim(convdim_outputs[-1],paddings_dec,kernels_dec,strides_dec)


model = VAE(args=args,
               in_channels=1,
               latent_dim=512,
               convdim_enc_outputs=convdim_outputs, 
               kernels=kernels_enc, 
               strides=strides_enc)

In [ ]:
mnist_data       = datasets.MNIST('', train=True, transform=transforms.ToTensor(), download=True)
mnist_dataloader = DataLoader(dataset=mnist_data, batch_size=4096, shuffle=True)

In [ ]:
batch, labels = next(iter(mnist_dataloader))
batch, labels = batch.to('cuda:0'), labels.to('cuda:0')

In [ ]:
labels.shape

In [ ]:
model = torch.load('/data/scratch/sarperyurtseven/results/training_results/deneme_vae/3/model_epoch-74.pt', map_location='cuda:0')

In [ ]:
mu, log = model.encode(batch)
z = model.reparameterize(mu, log)

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
z_embedded = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(z.detach().cpu().numpy())
z_embedded.shape

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(z_embedded[:, 0], z_embedded[:, 1], c=labels.detach().cpu().numpy(), cmap='brg')
plt.colorbar()
plt.show()

In [ ]:
model.encoder()

In [ ]:
z = torch.randn(10,512).to('cuda:0')

In [ ]:
x = model.decoder_input(z)
x = x.view(10,512,3,3)
x = model.decoder(x)
x = model.final_layer(x)

In [ ]:
x = x.detach().cpu().numpy()

In [ ]:
plt.imshow(np.transpose(x[1],(1,2,0)))

In [ ]:
kernels_enc = [7,7,7,7,7,7,7]
hidden_dims = [2**(i+3) for i in range(len(kernels_enc))]

In [ ]:
hidden_dims

In [ ]:
coron_psfs  = glob.glob('/data/scratch/bariskurtkaya/dataset/CORON_PSF_SAMPLES/*.fits')

In [ ]:
coron_psfs = sorted(coron_psfs)
corons   = []
for i in range(len(coron_psfs)):
    coron = fits.open(coron_psfs[i])[0].data
    corons.append(coron)

corons  = np.concatenate(np.expand_dims(corons, axis=0))


In [ ]:
bs = 4
nrows = 1
fig, axes = plt.subplots(nrows=nrows,ncols=bs,figsize=(15,10))

for i, (row,col) in enumerate(product(range(nrows),range(bs))):

    axes[col].imshow(corons[i])
    axes[col].set_yticks([])
    axes[col].set_xticks([])


plt.subplots_adjust(wspace=0,hspace=0)
plt.show()

In [ ]:
for i in range(len(coron_psfs)):
    coron = fits.open(coron_psfs[i])[0].data
    filter = coron_psfs[i].split('/')[-1].split('_')[0].split('-')[-1]
    print(f"Filter: {filter}  Max: {np.max(coron)}  Min: {np.min(coron)}")

In [ ]:
train_paths = glob.glob(os.path.join(INJECTIONS,'fc5_coron_injections_train/*.npy'))
print(len(train_paths))

syndata        = SynDataset(image_paths=train_paths)
syndata_loader = DataLoader(dataset=syndata, batch_size=8, shuffle=True)

In [ ]:
idx, batch = next(enumerate(syndata_loader))

In [ ]:
batch.shape

In [ ]:
for i in batch:
    print(f"Max: {torch.max(i)}  Min: {torch.min(i)}")
    print(f"Arcsinh Max: {torch.max(torch.arcsinh(i))}  Min: {torch.min(torch.arcsinh(i))}")
    print()

In [ ]:
bs = 4
nrows = 1
fig, axes = plt.subplots(nrows=nrows,ncols=bs,figsize=(15,10))

for i, (row,col) in enumerate(product(range(nrows),range(bs))):

    axes[col].imshow(batch[i][0])
    axes[col].set_yticks([])
    axes[col].set_xticks([])


plt.subplots_adjust(wspace=0,hspace=0)
plt.show()